In [27]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models

import sys
sys.path.insert(1, '../')
from msn.model import get_model
from configs.config import get_config

In [28]:
config = get_config()
config

callback_config:
  checkpoint_filepath: wandb/model_{epoch}
  early_patience: 6
  rlrp_factor: 0.2
  rlrp_patience: 3
  save_best_only: true
  use_earlystopping: true
  use_reduce_lr_on_plateau: false
  viz_num_images: 100
dataset_config:
  batch_size: 8
  num_crops:
  - 2
  - 5
  shuffle_buffer: 100
  size_crops:
  - 224
  - 96
  use_options: true
model_config:
  attention_probs_dropout_prob: 0.0
  backbone: ViT
  decoder_hidden_size: 512
  decoder_intermediate_size: 2048
  decoder_num_attention_heads: 16
  decoder_num_hidden_layers: 8
  dropout_rate: 0.5
  hidden_act: gelu
  hidden_dropout_prob: 0.0
  hidden_size: 768
  initializer_range: 0.02
  intermediate_size: 3072
  is_encoder_decoder: false
  layer_norm_eps: 1.0e-12
  mask_ratio: 0.75
  model_img_channels: 3
  model_img_height: 224
  model_img_width: 224
  norm_pix_loss: false
  num_attention_heads: 12
  num_channels: 3
  num_hidden_layers: 12
  patch_size: 16
  post_gap_dropout: false
  qkv_bias: true
seed: 0
train_config:
  e

In [29]:
tf.keras.backend.clear_session()
model = get_model(config)

In [30]:
model

In [31]:
pixel_values = tf.random.normal((8, 224, 224, 3))
print(model(pixel_values=pixel_values))

(<tf.Tensor: shape=(8, 50, 768), dtype=float32, numpy=
array([[[-0.57400256, -1.6440315 , -1.0521203 , ...,  1.1635206 ,
         -0.5148284 , -0.36135522],
        [-0.96005833, -1.0109295 , -0.4002926 , ...,  1.7490499 ,
         -0.4647035 ,  0.14602873],
        [ 0.01853287, -0.9749651 , -0.03791261, ...,  0.7811097 ,
         -0.77030885,  0.10929209],
        ...,
        [-0.8161026 , -0.96734303, -0.7000211 , ...,  1.3471062 ,
          0.5125636 ,  0.9819283 ],
        [-0.8642162 , -1.2996521 , -0.42890722, ...,  0.9295301 ,
         -0.04742081,  1.2941184 ],
        [-0.30199286, -0.77637637,  0.2577251 , ...,  0.9801136 ,
          0.08928126,  1.3520405 ]],

       [[-0.60744596, -1.7376442 , -1.0225233 , ...,  1.2539926 ,
         -0.59880984, -0.42464575],
        [-1.1899664 , -1.2062316 , -1.2620187 , ...,  1.0863484 ,
          0.5872282 ,  0.72593486],
        [ 0.18264773, -0.29886714,  0.05705667, ...,  1.1849644 ,
          0.35948834,  0.26277968],
        ...,

In [32]:
model.summary()

Model: "tf_vi_tmae_main_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embeddings (ViTMAEEmbedding  multiple                 742656    
 s)                                                              
                                                                 
 encoder (TFViTMAEEncoder)   multiple                  85054464  
                                                                 
 layernorm (LayerNormalizati  multiple                 1536      
 on)                                                             
                                                                 
Total params: 85,798,656
Trainable params: 85,647,360
Non-trainable params: 151,296
_________________________________________________________________
